In [22]:
# read new data
# read each page from excel file
import pandas as pd
import numpy as np

filename='cleaned'
Pacific=pd.read_excel(filename+'.xlsx',sheet_name='Pacific')
Atlantic=pd.read_excel(filename+'.xlsx',sheet_name='Atlantic')
Mediterranean=pd.read_excel(filename+'.xlsx',sheet_name='Mediterranean')
Southern=pd.read_excel(filename+'.xlsx',sheet_name='Southern Ocean')
Arctic=pd.read_excel(filename+'.xlsx',sheet_name='Arctic')

In [23]:

# we will be only merge one ocean at a time 

df2 = Pacific[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df2['Lat'] = df2['Lat'].astype(str)
df2['Long'] = df2['Long'].astype(str)

# here shockingly we have chinese character in the data!
df2['Lat'] = df2['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df2['Long'] = df2['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df2 = df2.dropna(subset=['Lat','Long'])

<ipython-input-23-e6a624ea674d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Lat'] = df2['Lat'].astype(str)
<ipython-input-23-e6a624ea674d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Long'] = df2['Long'].astype(str)
<ipython-input-23-e6a624ea674d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [24]:
import pandas as pd
import tarfile
import gzip

from pyhdf.SD import SD, SDC
import tempfile
import numpy as np
from scipy.interpolate import griddata

# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df2['Lat'].copy()
orig_lons = df2['Long'].copy()

for year in years:
    with tarfile.open(f'data/mld.hycom_030.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'mld' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df2['Lat'], df2['Long'])))
                        df2[f'mld_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df2['Year'] = year
        df2['Lat'] = orig_lats
        df2['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df2])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('mld_means_1.csv', index=False)

In [25]:
# add a column of yearly mean
all_results['yearly_mean_mld'] = all_results.filter(regex='mld_').mean(axis=1)

In [26]:
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Pacific['Cr_nmol/kg']

In [27]:
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Pacific'

In [28]:
# save the data csv
all_results.to_csv('mld_means_1.csv', index=False)

In [29]:
## check the data
df_mld_means = pd.read_csv('mld_means_1.csv')
df_mld_means.sample(10)

,Lat,Long,Year,mld_09,mld_10,mld_11,mld_12,mld_01,mld_02,mld_04,mld_05,mld_06,mld_07,mld_08,mld_03,yearly_mean_mld,Cr_nmol/kg,Ocean
11799,21.000000,-132.000000,2019,35.773823,44.033195,54.647850,61.697723,65.464615,39.998505,37.391120,24.333548,28.156792,32.953600,42.110462,45.117050,42.639860,NaN,Pacific
10350,50.000000,-145.000000,2016,17.160799,29.502502,45.377365,55.758110,62.890617,71.293230,51.501812,31.589933,21.348188,19.492756,15.737129,73.341610,41.249500,3.853659,Pacific
3013,-8.000000,-79.845000,2002,27.087984,24.019594,16.052288,15.315970,12.984103,11.577427,13.890297,17.621280,23.316814,36.388794,33.158220,11.862844,20.272968,1.500000,Pacific
1022,-12.010000,-79.200000,1998,30.703670,22.395218,17.953556,16.041576,18.049812,18.394129,19.967012,28.251144,45.978848,36.421505,39.396206,NaN,26.686607,3.460183,Pacific
10811,24.283333,-114.983333,2017,14.572323,14.941182,16.498848,23.960426,33.799854,26.152592,24.388525,21.035969,17.936180,12.862591,15.503565,22.272566,20.327051,4.355481,Pacific
8563,-14.000000,-99.000000,2012,62.235670,32.146027,42.556213,37.028057,23.676428,23.933200,28.135052,33.344200,31.751907,42.561295,54.609653,24.496094,36.372814,3.008331,Pacific
10179,-14.000000,-99.000000,2015,33.906470,44.592820,52.376740,27.468456,20.946709,24.963343,28.529898,29.470053,49.714134,46.478638,47.438572,24.496094,35.865160,4.851725,Pacific
4618,-3.583000,-85.833000,2005,28.474422,30.479767,20.338902,12.629570,11.993477,11.764564,19.533653,19.734050,24.645233,33.642160,27.701319,12.181180,21.093193,5.290000,Pacific
10181,-14.000000,-99.000000,2015,33.906470,44.592820,52.376740,27.468456,20.946709,24.963343,28.529898,29.470053,49.714134,46.478638,47.438572,24.496094,35.865160,5.088152,Pacific
9027,-34.670000,154.550000,2013,43.255870,44.462242,59.141174,33.416084,44.395340,46.464250,71.151460,71.964620,66.751560,63.897453,77.528030,52.580505,56.250713,3.027112,Pacific


In [30]:
# we will be only merge one ocean at a time 

df3 = Atlantic[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df3['Lat'] = df3['Lat'].astype(str)
df3['Long'] = df3['Long'].astype(str)

# here shockingly we have chinese character in the data!
df3['Lat'] = df3['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df3['Long'] = df3['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df3 = df3.dropna(subset=['Lat','Long'])


<ipython-input-30-a2adcc3e5c57>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Lat'] = df3['Lat'].astype(str)
<ipython-input-30-a2adcc3e5c57>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Long'] = df3['Long'].astype(str)
<ipython-input-30-a2adcc3e5c57>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [31]:
import pandas as pd
import tarfile
import gzip
from pyhdf.SD import SD, SDC
import tempfile
import numpy as np
from scipy.interpolate import griddata

# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df3['Lat'].copy()
orig_lons = df3['Long'].copy()

for year in years:
    with tarfile.open(f'data/mld.hycom_030.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df3['Lat'], df3['Long'])))
                        df3[f'mld_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df3['Year'] = year
        df3['Lat'] = orig_lats
        df3['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df3])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('mld_means_2.csv', index=False)

In [32]:
# add a column of yearly mean
all_results['yearly_mean_mld'] = all_results.filter(regex='mld_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Atlantic['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Atlantic'

In [33]:
# save the data csv
all_results.to_csv('mld_means_2.csv', index=False)

In [34]:
# we will be only merge one ocean at a time 

df4 = Mediterranean[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df4['Lat'] = df4['Lat'].astype(str)
df4['Long'] = df4['Long'].astype(str)

# here shockingly we have chinese character in the data!
df4['Lat'] = df4['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df4['Long'] = df4['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df4 = df4.dropna(subset=['Lat','Long'])


<ipython-input-34-aacb700961e0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Lat'] = df4['Lat'].astype(str)
<ipython-input-34-aacb700961e0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Long'] = df4['Long'].astype(str)
<ipython-input-34-aacb700961e0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [35]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df4['Lat'].copy()
orig_lons = df4['Long'].copy()

for year in years:
    with tarfile.open(f'data/mld.hycom_030.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df4['Lat'], df4['Long'])))
                        df4[f'mld_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df4['Year'] = year
        df4['Lat'] = orig_lats
        df4['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df4])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('mld_means_3.csv', index=False)

In [36]:
# add a column of yearly mean
all_results['yearly_mean_mld'] = all_results.filter(regex='mld_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Mediterranean['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Mediterranean'
# save the data csv
all_results.to_csv('mld_means_3.csv', index=False)

In [37]:
# we will be only merge one ocean at a time 

df5 = Southern[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df5['Lat'] = df5['Lat'].astype(str)
df5['Long'] = df5['Long'].astype(str)

# here shockingly we have chinese character in the data!
df5['Lat'] = df5['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df5['Long'] = df5['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df5 = df5.dropna(subset=['Lat','Long'])

<ipython-input-37-ba7fc8eba4aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Lat'] = df5['Lat'].astype(str)
<ipython-input-37-ba7fc8eba4aa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Long'] = df5['Long'].astype(str)
<ipython-input-37-ba7fc8eba4aa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [38]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df5['Lat'].copy()
orig_lons = df5['Long'].copy()

for year in years:
    with tarfile.open(f'data/mld.hycom_030.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df5['Lat'], df5['Long'])))
                        df5[f'mld_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df5['Year'] = year
        df5['Lat'] = orig_lats
        df5['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df5])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('mld_means_4.csv', index=False)

In [39]:
# add a column of yearly mean
all_results['yearly_mean_mld'] = all_results.filter(regex='mld_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Southern['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Southern'
# save the data csv
all_results.to_csv('mld_means_4.csv', index=False)

In [40]:
# we will be only merge one ocean at a time 

df6 = Arctic[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df6['Lat'] = df6['Lat'].astype(str)
df6['Long'] = df6['Long'].astype(str)

# here shockingly we have chinese character in the data!
df6['Lat'] = df6['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df6['Long'] = df6['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df6 = df6.dropna(subset=['Lat','Long'])

<ipython-input-40-49ca953795be>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Lat'] = df6['Lat'].astype(str)
<ipython-input-40-49ca953795be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Long'] = df6['Long'].astype(str)
<ipython-input-40-49ca953795be>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [41]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df6['Lat'].copy()
orig_lons = df6['Long'].copy()

for year in years:
    with tarfile.open(f'data/mld.hycom_030.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df6['Lat'], df6['Long'])))
                        df6[f'mld_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df6['Year'] = year
        df6['Lat'] = orig_lats
        df6['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df6])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('mld_means_5.csv', index=False)

In [42]:
# add a column of yearly mean
all_results['yearly_mean_mld'] = all_results.filter(regex='mld_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Arctic['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Arctic'
# save the data csv
all_results.to_csv('mld_means_5.csv', index=False)

In [43]:
dt1 = pd.read_csv('mld_means_1.csv')
dt2 = pd.read_csv('mld_means_2.csv')
dt3 = pd.read_csv('mld_means_3.csv')
dt4 = pd.read_csv('mld_means_4.csv')
dt5 = pd.read_csv('mld_means_5.csv')
merged_df = pd.concat([dt1, dt2, dt3, dt4, dt5])
merged_df.to_csv('mld_means_new.csv', index=False)


In [44]:
## check the data
df_mld_means = pd.read_csv('mld_means_new.csv')
df_mld_means.sample(10)

,Lat,Long,Year,mld_09,mld_10,mld_11,mld_12,mld_01,mld_02,mld_04,mld_05,mld_06,mld_07,mld_08,mld_03,yearly_mean_mld,Cr_nmol/kg,Ocean
8377,-9.999000,-79.134000,2012,21.604820,23.027210,18.861963,12.547065,13.067307,13.217509,16.491669,21.407473,23.174011,28.038462,24.560125,15.555095,19.296060,2.530000,Pacific
23948,-46.970000,141.930000,1999,103.057280,102.950455,87.721840,58.018356,51.132977,68.318474,98.596146,106.240220,98.213120,118.885390,134.858380,NaN,93.453880,2.880965,Southern
13442,21.476667,-126.595000,2022,25.172592,32.326077,53.720116,70.297440,47.489070,46.780205,49.123300,38.048140,31.922838,27.018564,23.418705,35.157295,40.039524,4.034170,Pacific
17138,47.243400,-9.331883,2005,28.482086,40.281395,69.153090,83.286580,152.804170,140.035580,42.953247,23.801641,15.054678,17.181686,18.275131,200.220250,69.294130,NaN,Atlantic
10320,-21.566667,-114.300000,2016,45.339752,22.453900,27.899988,22.177624,18.796390,24.617613,35.492325,42.559220,53.015453,54.544920,45.253880,31.760994,35.326004,6.500000,Pacific
2213,24.283333,-114.983333,2001,16.164707,21.872047,29.897419,35.636340,41.395610,32.337494,25.542423,26.592655,19.512148,13.433430,13.389800,31.833996,25.634005,2.471071,Pacific
18652,-2.962500,-25.614800,2010,54.603466,40.785103,52.917484,39.502914,28.011198,32.757680,31.228874,32.537100,39.552456,49.912037,49.075850,35.774060,40.554850,2.709797,Atlantic
26919,-53.583000,149.298000,2020,117.081860,124.322000,66.848570,54.147810,43.221714,51.402130,63.646240,82.055695,93.573060,97.394135,93.803894,61.848656,79.112144,3.758093,Southern
25136,-47.000000,141.900000,2007,162.623870,173.892870,97.866714,60.285126,42.603065,60.025870,63.591633,73.241970,118.419970,122.544014,159.943240,74.493140,100.794290,NaN,Southern
16874,12.743667,-28.501500,2004,14.589371,17.886166,15.857555,36.326126,44.873096,38.656600,34.757305,32.137444,25.819876,14.680635,16.787842,33.814102,27.182180,2.930142,Atlantic
